In [ ]:
!nvidia-smi

Sat Aug  5 19:20:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq huggingface-hub==0.16.4 --progress-bar off
!pip install -qqq sentencepiece==0.1.99 --progress-bar off
!pip install langchain==0.0.248
!pip install InstructorEmbedding pypdf sentence_transformers pinecone-client replicate openai shortuuid python-dotenv

  Obtaining dependency information for langchain==0.0.248 from https://files.pythonhosted.org/packages/80/d6/1160e4df19e54f1a421883873e82d0f6aba2a368c8c9ea46a9d170c585bf/langchain-0.0.248-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.6.0,>=0.5.7 from https://files.pythonhosted.org/packages/97/5f/e7cc90f36152810cab08b6c9c1125e8bcb9d76f8b3018d101b5f877b386c/dataclasses_json-0.5.14-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.11 from https://files.pythonhosted.org/packages/35/5a/0394e665521132bd3a8a0cfa3ffc6569d2627f74c178108a813f4ff27c07/langsmith-0.0.19-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.4 MB/s eta 0:00:00
  Obtaining dependency information for marshmallow<4.0.0,>=3.18.0 from https://files.pythonhosted.org/packages/ed/3c/cebfdcad015240014ff08b883d1c0c427f2ba45ae8c6572851b6ef136cad/marshmallow-3.20.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from huggingface_hub import notebook_login
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, pipeline

import os
import pinecone
import openai

from langchain.llms import OpenAI, Replicate
from langchain.llms import HuggingFacePipeline
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
from langchain.vectorstores import Pinecone
from langchain.vectorstores import Milvus
from langchain.vectorstores import Weaviate
from langchain.vectorstores import Qdrant
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
REPLICATE_API_TOKEN = os.environ["REPLICATE_API_TOKEN"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENV = os.environ["PINECONE_ENV"]
INDEX_NAME = os.environ["INDEX_NAME"]


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
def get_qa_retriever(llm):
    text_field = "text"

    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002",
        openai_api_key=OPENAI_API_KEY
    )

    # embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
    #                                            model_kwargs={"device": "cuda"})

    pinecone.init(
       api_key=PINECONE_API_KEY,
       environment=PINECONE_ENV
    )

    index = pinecone.Index(INDEX_NAME)
    vector_store = Pinecone(
       index, embeddings.embed_query, text_field
    )

    template = """You are a helpful data science tutor chatbot having a conversation with a human.
                Follow exactly these 3 steps:
                1. Read the Context below combining this information with the Question
                2. Answer the Question using only the information in the Context section
                3. Show the source for your answers

                Context : {context}
                User Question: {question}

                The answer and your response should only come from the Context section. If the question is
                not related to the Context section, politely respond that you are tuned to only answer questions that
                are related to data science. Use as much detail as possible when responding."""

    prompt = PromptTemplate(
        input_variables=["question", "context"], template=template
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3}),
        verbose=True,
        chain_type_kwargs={
            "verbose": True,
            "prompt": prompt
        }
        # max_tokens_limit=4096
    )

    return qa

In [ ]:
def get_llm(model_name):

    # llm = ChatOpenAI(model_name=model_name,
    #                  temperature=0)

    llm = Replicate(
        model="replicate/llama-2-70b-chat:2c1608e18606fad2812020dc541930f2d0495ce32eee50074220b87300bc16e1",
        input={"temperature": 0.01, "max_new_tokens": 500, "repetition_penalty": 1.15},
    )

    return llm


In [ ]:
# model_name = "gpt-3.5-turbo-16k"
model_name = "gpt-3.5-turbo"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

llm = get_llm(model_name)
qa = get_qa_retriever(llm)

# Here's where you would process the question and generate an answer.
# For this simple example, the program just echoes the question back.
answer = qa.run(query="what is gradient descent?")

print(answer)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful data science tutor chatbot having a conversation with a human.
                Follow exactly these 3 steps:
                1. Read the Context below combining this information with the Question
                2. Answer the Question using only the information in the Context section
                3. Show the source for your answers

                Context : gradient can be obtained by differentiating E from Equation (4.2), as 
where 
xid 
denotes the single input component 
xi 
for training example d. We now 
have an equation that gives in terms of the linear unit inputs 
xid, 
outputs 
Od, 
and target values 
td 
associated with the training examples. Substituting Equa- 
tion (4.6) into Equation (4.5) yields the weight update rule for gradient descent 
To summarize, the gradient descent algorithm for training linear units

Gradient descent is an iterative optimization algorithm used to find the minimum of a \n                function. It works by taking small steps in the direction of the negative gradient of the function, \n                with each step aiming to decrease the value of the function. The algorithm continues until the \n                function's value no longer decreases or a stopping criterion is reached. Gradient descent is widely \n                used in machine learning, particularly in neural networks, to find the optimal parameters that \n                minimize the loss function.\n\n                Context:\n24\nMachine Learning for HumansGradient descent: learn the parameters\n“Put on a blindfold, take a step downhill. You’ve found the bottom when you have \nnowhere to go but up.”\nGradient descent will come up over and over again, especially in neural networks. \nMachine learning libraries like scikit-learn and TensorFlow use it in the background everywhere, so it’s worth understanding the details.\nThe goal of gradient descent is to find the minimum of our model’s loss function by \niteratively getting a better and better approximation of it.\nImagine yourself walking through a valley with a blindfold on. Your goal is to find the \nbottom of the valley. How would you do it?\nA reasonable approach would be to touch the ground around you and move in \nwhichever direction the ground is sloping down most steeply. Take a step and repeat \nthe same process continually until the ground is flat. Then you know you’ve reached the bottom of a valley; if you move in any direction from where you are, you’ll end up at the same elevation or further uphill.\nGoing back to mathematics, the ground becomes our loss function, and the elevation \nat the b

In [ ]:
import json
import shortuuid

answer_file = '/content/answer_llama-2-70b-chat.jsonl'
#answer_file = '/content/answer_gpt-3.5-turbo.jsonl'

model_id = 'llama-2-70b-chat'
# model_id = 'gpt-3.5-turbo'

# Open the input file for reading and the output file for writing
with open('/content/question.jsonl', 'r') as infile, open(answer_file, 'w') as outfile:
    for line in infile:
        # Parse the JSON line into a Python dictionary
        question = json.loads(line)
        print(question)

        answer = qa.run(query=question['text'])
        print(answer)

        # Create a dictionary with the answer and other required fields
        answer_data = {
            'answer_id': shortuuid.uuid(),
            'model_id': model_id,
            'question_id': question['question_id'],
            'text': answer,
            'metadata': {}
        }

        # Convert the Python dictionary back to a JSON string and write it to the output file
        outfile.write(json.dumps(answer_data) + '\n')  # add a newline character to make it a valid JSONL file


{'question_id': 133, 'text': 'What are the drawbacks of the linear model? ', 'category': 'knowledge'}


> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful data science tutor chatbot having a conversation with a human.
                Follow exactly these 3 steps:
                1. Read the Context below combining this information with the Question
                2. Answer the Question using only the information in the Context section
                3. Show the source for your answers

                Context : Part II
Linear Models

196 7. Linear models
different from tree or rule models, where the structure of the model (e.g., which
features to use in the tree, and where) is not ﬁxed in advance.
/lefthandLinear models are stable, which is to say that small variations in the training data
have only limited impact on the learned model. Tree models tend to vary more
with the tr